In [64]:
import os, re, html
import pandas as pd
from nltk import sent_tokenize
from tqdm import tqdm

In [3]:
root_dir = '/media/dmlab/My Passport/DATA/ComBERT/data'
item1s_filepath = os.path.join(root_dir, 'Item1s_2016_by_sec_api_ExtractorApi_with_cik_of_sec_cik_mapper_3990.csv')
company_filepath = os.path.join(root_dir, 'company_info_sec_cik_mapper_12057_20220802.csv')

In [51]:
company_df = pd.read_csv(company_filepath)
company_df = company_df.astype({"CIK": int}, errors='raise')
company_df.head()

,CIK,Ticker,Name,Exchange
0,1750,AIR,Aar Corp,NYSE
1,1800,ABT,Abbott Laboratories,NYSE
2,1961,WDDD,Worlds Inc,OTC
3,2098,ACU,Acme United Corp,NYSE
4,2178,AE,"Adams Resources & Energy, Inc.",NYSE


In [54]:
company_df[company_df['Ticker'].isin(['AMZN', 'GOOG', 'GOOGL'])]

,CIK,Ticker,Name,Exchange
2572,1018724,AMZN,Amazon Com Inc,Nasdaq
7389,1652044,GOOG,Alphabet Inc.,Nasdaq
7390,1652044,GOOGL,Alphabet Inc.,Nasdaq


In [52]:
item_df = pd.read_csv(item1s_filepath)
item_df = item_df.astype({"cik": int}, errors='raise')
item_df.head()

,accessionNo,cik,companyNameLong,ticker,description,formType,type,filingUrl,filedAt,item_1
0,0001047469-16-014299,1750,AAR CORP (AIR) (CIK 0000001750),AIR,10-K,10-K,10-K,https://www.sec.gov/Archives/edgar/data/1750/0...,2016-07-13,ITEM 1. BUSINESS (Dollars in millions) \n\nGe...
1,0001047469-16-010246,1800,ABBOTT LABORATORIES (ABT) (CIK 0000001800),ABT,10-K,10-K,10-K,https://www.sec.gov/Archives/edgar/data/1800/0...,2016-02-19,ITEM 1. BUSINESS \n\nGENERAL DEVELOPMENT OF B...
2,0001264931-16-000339,1961,WORLDS INC (WDDD) (CIK 0000001961),WDDD,NaN,10-K,10-K,https://www.sec.gov/Archives/edgar/data/1961/0...,2016-04-13,ITEM 1. BUSINESS. \n\n&#160;\n\nGeneral \n\n&...
3,0001026608-16-000091,2098,ACME UNITED CORP (ACU) (CIK 0000002098),ACU,10-K,10-K,10-K,https://www.sec.gov/Archives/edgar/data/2098/0...,2016-03-11,Item 1. Business \n\n&#160; \n\n&#160; \n\nOv...
4,0000002178-16-000064,2178,"ADAMS RESOURCES & ENERGY, INC. (AE) (CIK 00000...",AE,"FORM 10-K 151231 ADAMS RESOURCES & ENERGY, INC.",10-K,10-K,https://www.sec.gov/Archives/edgar/data/2178/0...,2016-03-11,Items 1 and 2. BUSINESS AND PROPERTIES \n\nBu...


In [89]:
# company_df: sec-cik-mapper 기반 기업 정보
# item_df: company_df에 기초하여 URL을 수집한 결과
# company_df에는 없는 CIK가 item_df에만 있다는 게 말이 안됨. SEC-API 수집 과정 중 발생한 에러인 듯
print(company_df[company_df['CIK'].isin([1581374, 1081316, 1003410, 1034563, 55387])])

item_df[item_df['cik'].isin([1581374, 1081316, 1003410, 1034563, 55387])]['companyNameLong'].values

Empty DataFrame
Columns: [CIK, Ticker, Name, Exchange]
Index: []


array(['Scotiabank Covered Bond Guarantor Limited Partnership (CIK 0001581374)',
       'BERKSHIRE HATHAWAY ENERGY CO (CIK 0001081316)',
       'DUKE REALTY LIMITED PARTNERSHIP/ (CIK 0001003410)',
       'ICAHN ENTERPRISES HOLDINGS L.P. (CIK 0001034563)',
       'KENTUCKY UTILITIES CO (CIK 0000055387)'], dtype=object)

In [12]:
# Amazon: 1018724
doc = item_df[item_df['cik']==1018724].iloc[0]['item_1']
doc[:1000]

' Item 1. \n\nBusiness \n\nThis Annual Report on Form 10-K and the documents incorporated herein by reference contain forward-looking statements based on expectations, estimates, and projections as of the date of this filing. Actual results may differ materially from those expressed in forward-looking statements. See Item 1A of Part I&#8212;&#8220;Risk Factors.&#8221; \n\nAmazon.com, Inc. was incorporated in 1994 in the state of Washington and reincorporated in 1996 in the state of Delaware. Our principal corporate offices are located in Seattle, Washington. We completed our initial public offering in May 1997 and our common stock is listed on the NASDAQ Global Select Market under the symbol &#8220;AMZN.&#8221; \n\nAs used herein, &#8220;Amazon.com,&#8221; &#8220;we,&#8221; &#8220;our,&#8221; and similar terms include Amazon.com, Inc. and its subsidiaries, unless the context indicates otherwise. \n\nGeneral \n\nAmazon.com opened its virtual doors on the World Wide Web in July 1995. We 

### Document -> Sentences

In [39]:
removal_list =  "‘, ’, ◇, ‘, ”,  ’, ', ·, \“, ·, △, ●,  , ■, (, ), \", >>, `, /, -,∼,=,ㆍ<,>, .,?, !,【,】, …, ◆,%"
for sent in sent_tokenize(doc):
    print('[Step {}] {}'.format(0, sent))
    
    sent = html.unescape(sent)
    print('[Step {}] {}'.format(1, sent))
    
    sent = sent.translate(str.maketrans(removal_list, ' '*len(removal_list)))
    print('[Step {}] {}'.format(2, sent))
    
    sent = re.sub("\s+", " ", sent)
    print('[Step {}] {}'.format(3, sent))
    
    sent = sent.lower()
    print('[Step {}] {}\n'.format('Final', sent))

[Step 0]  Item 1. 

Business 

This Annual Report on Form 10-K and the documents incorporated herein by reference contain forward-looking statements based on expectations, estimates, and projections as of the date of this filing.
[Step 1]  Item 1. 

Business 

This Annual Report on Form 10-K and the documents incorporated herein by reference contain forward-looking statements based on expectations, estimates, and projections as of the date of this filing.
[Step 2]  Item 1  

Business 

This Annual Report on Form 10 K and the documents incorporated herein by reference contain forward looking statements based on expectations  estimates  and projections as of the date of this filing 
[Step 3]  Item 1 Business This Annual Report on Form 10 K and the documents incorporated herein by reference contain forward looking statements based on expectations estimates and projections as of the date of this filing 
[Step Final]  item 1 business this annual report on form 10 k and the documents incorpo

### subnames of a company name

In [41]:
def subnames_of_company_name(fullname): 
    strings = fullname.split(' ')
    subnames = [' '.join(strings[:i+1]) for i in range(len(strings))] # ['amazon', 'amazon com', 'amazon com inc' ]
    subnames.reverse() # ['amazon com inc', 'amazon com', 'amazon']
    subnames.extend(['we', 'the company'])    
    subnames = [item.strip() for item in subnames if item.strip() != '']
    return subnames

fullname = 'Amazon Com Inc'.lower()
subnames = subnames_of_company_name(fullname)
subnames

['amazon com inc', 'amazon com', 'amazon', 'we', 'the company']

In [44]:
def replace_subnames_to_target(sent, subnames, target):
    for sub in subnames:
        if sub in sent:
            sent = sent.replace(sub, '[TO-BE-REPLACED]')
    sent = sent.replace('[TO-BE-REPLACED]', target)
    return sent

sent = "we completed our initial public offering in may 1997 and our common stock is listed on the nasdaq global select market under the symbol amzn as used herein amazon com we our and similar terms include amazon com inc and its subsidiaries unless the context indicates otherwise "
sent = replace_subnames_to_target(sent, subnames, fullname)
sent

'amazon com inc completed our initial public offering in may 1997 and our common stock is listed on the nasdaq global select market under the symbol amzn as used herein amazon com inc amazon com inc our and similar terms include amazon com inc and its subsidiaries unless the context indicates otherwise '

# Replace names

In [69]:
removal_list =  "‘, ’, ◇, ‘, ”,  ’, ', ·, \“, ·, △, ●,  , ■, (, ), \", >>, `, /, -,∼,=,ㆍ<,>, .,?, !,【,】, …, ◆,%"
def clean(sent):
    sent = html.unescape(sent)
    sent = sent.translate(str.maketrans(removal_list, ' '*len(removal_list)))
    sent = re.sub("\s+", " ", sent)
    sent = sent.lower()
    return sent

def trim(sent):
    sent = re.sub("\s+", " ", sent)
    sent = sent.strip()
    return sent

def get_fullname(cik):
    try:
        return company_df[company_df['CIK']==cik].iloc[0]['Name']
    except:
        print('No full name for cik {}'.format(cik))
        return None
fullname = get_fullname(1018724).lower()
fullname

'amazon com inc'

In [75]:
for cik, doc in item_df[['cik', 'item_1']].values[:10]:
    fullname = get_fullname(cik)
    if fullname is None: 
        continue
        
    fullname = fullname.lower()
    for sent in sent_tokenize(doc):
        sent = clean(sent)
        sent = replace_subnames_to_target(sent, subnames_of_company_name(fullname), fullname)

        # 원 텍스트에 기업명이 등장한 케이스만 post-training에 사용
        if fullname in sent: 
            sent = trim(sent)
            print('[{}] {}\n'.format(fullname, sent))

[aar corp] item 1 business dollars in millions general aar corp and its subsidiaries are referred to herein collectively as aar corp company aar corp us and our unless the context indicates otherwise

[aar corp] aar corp was founded in 1951 organized in 1955 and reincorporated in delaware in 1966

[aar corp] aar corp are a diversified provider of products and services to the worldwide aviation and government and defense markets

[aar corp] during fiscal 2015 aar corp executed on a comprehensive strategic plan that included: • the sale of our telair cargo group for cash of $714 million resulting in pre tax gains of $198 6 million in the fourth quarter of fiscal 2015 and $27 7 million in the first quarter of fiscal 2016 from the receipt of contingent consideration ; • a decision to divest our precision systems manufacturing business; • the exit of certain product lines and inventories in our aviation services businesses that aar corpre underperforming or not part of our strategy going fo

[ceco environmental corp] item 1 business general ceco environmental corp and its consolidated subsidiaries ceco environmental corp ceco environmental corp ceco environmental corp us or our is a diversified global provider of leading highly engineered technologies to the environmental energy and fluid handling and filtration industrial segments targeting specific niche focused end markets through an attractive asset light business model strategically balanced across the world

[ceco environmental corp] ceco environmental corp has over $5 billion of installed equipment base with end users which ceco environmental corp target to expand and grow a higher recurring revenue of aftermarket products and services

[ceco environmental corp] ceco environmental corp s ceco environmental corpll respected brands technologies and solutions have been evolving for ceco environmental corpll over 50 years to become leading edge technologies in specific niche global end markets

[ceco environmental corp]